In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ObscureSuperhero').getOrCreate()

In [2]:
# read connection graph
lines = spark.read.text('file:////Users/giovanna/Documents/GitHub/pyspark/SparkCourse/Marvel-graph.txt')

In [3]:
lines.show(1)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
+--------------------+
only showing top 1 row



In [4]:
from pyspark.sql import functions as F

In [5]:
sdf_hero_total = lines.withColumn('hero', F.split(F.col('value'), ' ')[0])\
                        .withColumn('total_conn', F.size(F.split(F.col('value'), ' '))-1)\
                        .groupby('hero')\
                            .agg(F.sum('total_conn').alias('total_conn'))
sdf_hero_total.show(1)

+----+----------+
|hero|total_conn|
+----+----------+
| 691|         7|
+----+----------+
only showing top 1 row



In [6]:
# sdf_hero_total.select(F.min('total_conn'))[0] -- assim é possível filtrar o DataFrame
min_conn = sdf_hero_total.agg(F.min('total_conn')).first()[0] # -- assim é possível obter o valor diretamente

In [7]:
# Filter obscure
sdf_hero_total = sdf_hero_total.filter(F.col('total_conn') == min_conn)

In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([StructField('id', IntegerType(), True),\
                        StructField('name', StringType(), True)])
names = spark.read.schema(schema=schema).option('sep',' ').csv('file:////Users/giovanna/Documents/GitHub/pyspark/SparkCourse/Marvel-names.txt')
names.show(4)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
+---+--------------------+
only showing top 4 rows



In [9]:
sdf_hero_total = sdf_hero_total.join(names, sdf_hero_total.hero == names.id, 'left')

In [10]:
sdf_hero_total.show(10)

+----+----------+----+--------------------+
|hero|total_conn|  id|                name|
+----+----------+----+--------------------+
| 467|         1| 467|        BERSERKER II|
| 577|         1| 577|              BLARE/|
|3490|         1|3490|MARVEL BOY II/MARTIN|
|3489|         1|3489|MARVEL BOY/MARTIN BU|
|2139|         1|2139|      GIURESCU, RADU|
|1089|         1|1089|       CLUMSY FOULUP|
|1841|         1|1841|              FENRIS|
|4517|         1|4517|              RANDAK|
|5028|         1|5028|           SHARKSKIN|
| 835|         1| 835|     CALLAHAN, DANNY|
+----+----------+----+--------------------+
only showing top 10 rows

